# Pipeline


In [2]:
import pandas as pd
import requests
import numpy as np
import math
import pickle
import glob
import time 

In [2]:
def cambiar_minusculas_variable(archivo,variable):
    archivo[variable]=archivo[variable].str.lower()
start = time.time()

In [3]:

#path = '../data' # use your path
#all_files = glob.glob(path + "/*.zip")
#all_files = all_files[0:3]

#li = []
#i = 0
#for filename in all_files:
#    df = pd.read_csv(filename, index_col=None, header=0)
#    print(i)
#    i+=1
#    li.append(df)

#data1 = pd.concat(li, axis=0, ignore_index=True)

In [4]:

#path = '../data' # use your path
#all_files = glob.glob(path + "/*.zip")
#all_files = all_files[3:6]

#li = []
#i = 0
#for filename in all_files:
#    df = pd.read_csv(filename, index_col=None, header=0)
#    print(i)
#    i+=1
#    li.append(df)

#data2 = pd.concat(li, axis=0, ignore_index=True)

In [5]:

path = '../data' # use your path
all_files = glob.glob(path + "/*.zip")
all_files = all_files[6:9]

li = []
i = 0
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(i)
    i+=1
    li.append(df)

data3 = pd.concat(li, axis=0, ignore_index=True)

In [6]:

path = '../data' # use your path
all_files = glob.glob(path + "/*.zip")
all_files = all_files[9:12]

li = []
i = 0
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(i)
    i+=1
    li.append(df)

data4 = pd.concat(li, axis=0, ignore_index=True)

0
1
2


In [7]:
data = pd.concat([
    #data1,data2,
                  data3,data4], axis=0, ignore_index=True)
#data = pd.concat([data1,data2,data3,data4], axis=0, ignore_index=True)
print(data.shape)

In [8]:
#data = pd.read_csv('../data/201912-citibike-tripdata.csv.zip')

In [9]:
#Creación de la columna trip_id

#generamos los ID's
n = len(data.index)
digits_range = range(1345,n + 1345)
digits_list = list(digits_range)

#Agreagamos la columna al df
data['trip_id'] = digits_list

In [10]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_c = data.loc[data['usertype'] == 'Customer']

#Se obtienen las filas que tienen como usertype "Subscriber"
subset_s = data.loc[data['usertype'] == 'Subscriber']

In [11]:
subset_c.shape[0]

#Número de pases de 1 día
single_trip = round(subset_c.shape[0]*.684)

#Número de pases de 3 días
one_day = subset_c.shape[0] - single_trip

print("Numero de pases en diciembre de 1 día",one_day )
print("Numero de pases en diciembre de un viaje sencillo",single_trip )
one_day + single_trip

Numero de pases en diciembre de 1 día 24529
Numero de pases en diciembre de un viaje sencillo 53096


77625

In [12]:

#Creación de columna auxiliar para separar los pases de Costumer en 2
c_n = len(subset_c.index)
subset_c['aux']=np.random.binomial(1,.684,c_n)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
c_n


77625

In [14]:
#Creación de columna con el nombre de los pases nuevos
subset_c['type_pass'] = np.where(subset_c['aux'] == 1, "single_trip", "one_day")
#Borramos la columa auxiliar
del subset_c['aux']

subset_s['type_pass']='annual'
#Juntamos los dos dataframes
data = subset_s.append(subset_c, ignore_index=True)
data.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(955210, 17)

In [15]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_s = data.loc[data['type_pass'] == 'single_trip']

subset_s['trip_category']='one-way'

subset_o = data.drop(subset_s.index)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
#Creación de una columna nueva : trip_category
o_n = len(subset_o)
subset_o['aux']=np.random.binomial(1,.192,o_n)

subset_o['trip_category'] = np.where(subset_o['aux'] == 1, "round-trip", "one-way")
#Borramos la columa auxiliar
del subset_o['aux']

In [17]:
#Juntamos los dos dataframes
data = subset_s.append(subset_o, ignore_index=True)
data.shape

(955210, 18)

In [18]:
#Convertimos la duración a minutos
data['duration'] = round(data['tripduration']/60)

In [19]:
data['starttime'] = pd.to_datetime(data['starttime']).dt.strftime('%Y-%m-%d %H:%M')
data['stoptime'] = pd.to_datetime(data['stoptime']).dt.strftime('%Y-%m-%d %H:%M')

In [20]:
data_ = data.copy()

In [21]:
# remove spaces in column names
data_.columns = data_.columns.str.replace(' ', '_')

In [22]:
# Change to lower categorical variables
cambiar_minusculas_variable(data_,'start_station_name')
cambiar_minusculas_variable(data_,'end_station_name')
cambiar_minusculas_variable(data_,'usertype')
cambiar_minusculas_variable(data_,'type_pass')
cambiar_minusculas_variable(data_,'trip_category')

In [23]:
# Converting start and end times to datetime objects.
data_['starttime'] = pd.to_datetime(data_['starttime'])
data_['stoptime'] = pd.to_datetime(data_['stoptime'])

In [24]:
# Create label for rides that were charged based on current pricing plan.
def label_charged(duration):
    """"Function to create a new catergorical variable that indicate if theres has to be an adittional charge"""
    if duration <= 30:
        return 0
    else:
        return 1

data_['charged_ride'] = data_['duration'].apply(label_charged)

In [25]:
# Column for hour of day of ride.
data_['start_hour'] = data_['starttime'].dt.hour

In [26]:
latest_ride_date = data_['stoptime'].max()
latest_ride_date

Timestamp('2020-01-17 20:00:00')

In [27]:
bike_last_used_id = bike_last_used = data_.groupby('bikeid')['stoptime'].max().keys().tolist()
bike_last_used_date = data_.groupby('bikeid')['stoptime'].max().tolist()

In [28]:
# Number of bikes active each month in the past year.
for month in range(1,13):
    print(month, len([record for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
                      record[1] > latest_ride_date + np.timedelta64(-month, 'M') ]))

1 12454
2 13816
3 13816
4 13816
5 13816
6 13816
7 13816
8 13816
9 13816
10 13816
11 13816
12 13816


In [29]:
# Number of inactive bikes (bikes that haven't been used in the past 6 months)
len([record for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
     record[1] < latest_ride_date + np.timedelta64(-6, 'M') ])

0

In [30]:
# Create feature indicating active bikes, used in past 6 months. 1 if active, 0 if inactive.
inactive_ids = [record[0] for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
                record[1] < latest_ride_date + np.timedelta64(-6, 'M') ]
data_['active_6m'] = data_['bikeid'].apply(lambda bike_id: 0 if bike_id in (inactive_ids) else 1)

In [31]:
#Select variables to modeling
data_ = data_ [["trip_id", "duration","starttime", "stoptime", "trip_category", "type_pass"]]

In [32]:
del data
data_model = data_
del data_

In [33]:
# Create dummy variables for passholder type.
one_hot_pass = pd.get_dummies(data_model['type_pass'])

# Create dummy variables for trip route category.
one_hot_trip_type = pd.get_dummies(data_model['trip_category'])

data_model = data_model.join([one_hot_pass, one_hot_trip_type])

In [34]:
# Creates feature for 30-45 minute blocks that walk-up users have to pay for.
# Creates another feature for 15 minute blocks for passholders that exclude first 30 or 45 minute block.
data_model['time_block_count'] = data_model['duration'].apply(lambda row: \
                                                                        math.ceil(data_model['duration'][row]/45) if data_model['type_pass'][row] == 'annual' 
                                                                        else math.ceil(data_model['duration'][row]/30))
data_model['time_block_count_post_free'] = data_model['duration'].apply(lambda row: \
                                                                        max(math.ceil(data_model['duration'][row]/15)-3,0) if data_model['type_pass'][row] == 'annual' 
                                                                        else max(math.ceil(data_model['duration'][row]/15)-2,0))

In [35]:
from scipy import stats
data_model = data_model[(np.abs(stats.zscore(data_model['time_block_count_post_free'])) < 3)]

In [36]:
pass_groups = data_model.groupby('type_pass')
pass_groups.groups.keys()

agg_pass_groups = pass_groups.agg({
    'trip_id' : 'count',
    'time_block_count' : 'sum',
    'time_block_count_post_free' : 'sum',
    'duration' : 'sum'
}).rename(columns = {
    'trip_id' : 'total_trips',
    'time_block_count' : 'total_free_blocks',
    'time_block_count_post_free' : 'total_15min_blocks_post_free',
    'duration' : 'total_minutes'
}).reset_index()

agg_pass_groups['type_pass'] = ['single','annualy','daily']
agg_pass_groups.index = agg_pass_groups['type_pass']
agg_pass_groups = agg_pass_groups.drop('type_pass',1)
agg_pass_groups

,total_trips,total_free_blocks,total_15min_blocks_post_free,total_minutes
type_pass,,,,
single,875156,1103363,310605,10427110.0
annualy,24236,28353,5538,801920.0
daily,52821,61562,11822,1868049.0


In [37]:
agg_pass_groups.to_csv('blocks')

In [38]:

### ASSUMPTIONS
# Passes sold = Total rides / Rides to break even
price_wo_pass = 3.5
price_w_pass_day = 0
price_w_pass_annual = 0

# Average difference between passholders and walk-up riders across previous and current pricing.
# This is represented by the difference in prices per 30 minute block since pass holders get the first 30 minutes free.
# As current data is from before the price change, previous prices are used to estimate number of passes.

day_diff = price_wo_pass - price_w_pass_day
annual_diff = price_wo_pass - price_w_pass_annual

# Prices of passes.

day_pass_price = 15
annual_pass_price = 180

# Estimated rides required for breakeven.

rides_day_pass_estimate = day_pass_price / day_diff
rides_annual_pass_estimate = annual_pass_price / annual_diff

print(" Rides per day required for daily pass:",rides_day_pass_estimate,'\n', 
      "Rides per year required for annual pass:", rides_annual_pass_estimate)

 Rides per day required for daily pass: 4.285714285714286 
 Rides per year required for annual pass: 51.42857142857143


In [39]:
rides_day_pass_estimate *= 1.4

rides_annual_pass_estimate *= 1.4

print("Rides per day required for daily pass (scaled):",rides_day_pass_estimate,'\n', 
      "Rides per year required for annual pass (scaled):", rides_annual_pass_estimate)

Rides per day required for daily pass (scaled): 5.999999999999999 
 Rides per year required for annual pass (scaled): 72.0


In [40]:
# Divide total trips by rides required for breakeven per pass duration (day/month/year) to get estimated number
# of passholders.

passes_sold_day = math.ceil(agg_pass_groups.loc['daily','total_trips']/rides_day_pass_estimate)
passes_sold_annual = math.ceil(agg_pass_groups.loc['annualy','total_trips']/rides_annual_pass_estimate)
passes_sold_total = passes_sold_day + passes_sold_annual

agg_pass_groups['passes_sold'] = [0, passes_sold_annual, passes_sold_day]

print("Estimated passes sold for daily pass:",passes_sold_day,'\n', 
      "Estimated passes sold for annual pass:", passes_sold_annual,'\n',
     "Total passes sold:", passes_sold_total)

Estimated passes sold for daily pass: 8804 
 Estimated passes sold for annual pass: 337 
 Total passes sold: 9141


In [34]:
#Prueba de total total unimodularity para las restricciones del problema

matrix_2_1 = np.array([[1,0],[0,1]])
matrix_2_2 = np.array([[0,1],[0,0]])
matrix_2_3 = np.array([[0,0],[1,1]])
matrix_2_4 = np.array([[1,1],[0,0]])
matrix_2_5 = np.array([[0,0],[0,0]])
matrix_2_6 = np.array([[0,0],[1,0]])

matrix_3_1 = np.array([[1,0,0],[0,1,1],[0,0,0]])
matrix_3_2 = np.array([[0,0,0],[1,1,0],[0,0,1]])
matrix_3_3 = np.array([[0,0,0],[1,0,0],[0,1,1]])
matrix_3_4 = np.array([[1,0,0],[0,1,0],[0,0,1]])
matrix_3_5 = np.array([[0,0,1],[0,0,0],[1,1,0]])
matrix_3_6 = np.array([[0,1,0],[0,0,1],[1,0,0]])

det_2_1 = np.linalg.det(matrix_2_1)
det_2_2 = np.linalg.det(matrix_2_2)
det_2_3 = np.linalg.det(matrix_2_3)
det_2_4 = np.linalg.det(matrix_2_4)
det_2_5 = np.linalg.det(matrix_2_5)
det_2_6 = np.linalg.det(matrix_2_6)

det_3_1 = np.linalg.det(matrix_3_1)
det_3_2 = np.linalg.det(matrix_3_2)
det_3_3 = np.linalg.det(matrix_3_3)
det_3_4 = np.linalg.det(matrix_3_4)
det_3_5 = np.linalg.det(matrix_3_5)
det_3_6 = np.linalg.det(matrix_3_6)

determinante = []
determinante.append(det_2_1)
determinante.append(det_2_2)
determinante.append(det_2_3)
determinante.append(det_2_4)
determinante.append(det_2_5)
determinante.append(det_2_6)
determinante.append(det_3_1)
determinante.append(det_3_2)
determinante.append(det_3_3)
determinante.append(det_3_4)
determinante.append(det_3_5)
determinante.append(det_3_6)

j=0
for i in determinante:
    if (i==-1 or i==0 or i==1):
        j=j+1

if j==len(determinante):
    print("Total unimodularity matrix")
else:
    print('No total unimodularity matrix ')

Total unimodularity matrix


### Escenario con precios actuales

In [82]:
agg_pass_groups['new_pass_prices'] = [0, 180, 15]
agg_pass_groups['average_charged_blocks_per_ride'] = agg_pass_groups['total_15min_blocks_post_free'] / agg_pass_groups['total_trips']
agg_pass_groups['percent_total_trips'] = agg_pass_groups['total_trips']/sum(agg_pass_groups['total_trips'])
agg_pass_groups

,total_trips,total_free_blocks,total_15min_blocks_post_free,total_minutes,passes_sold,new_pass_prices,average_charged_blocks_per_ride,percent_total_trips
type_pass,,,,,,,,
single,875156,1103363,310605,10427110.0,0,0,0.354914,0.919076
annualy,24236,28353,5538,801920.0,337,180,0.228503,0.025452
daily,52821,61562,11822,1868049.0,8804,15,0.223812,0.055472


In [83]:
current_tot_rev = (4 * agg_pass_groups['total_15min_blocks_post_free']['daily']
                   + 2.7 * agg_pass_groups['total_15min_blocks_post_free']['single']
                   + 1.8 * agg_pass_groups['total_15min_blocks_post_free']['annualy']
                   + sum(agg_pass_groups['passes_sold'] * agg_pass_groups['new_pass_prices']))
print('Total revenue: $', current_tot_rev)

Total revenue: $ 1088609.9


In [84]:
#Se declaran los precios
previous_walk_up = 3.5
previous_pass_daily = 2.7
previous_pass_annualy = 1.8

lhs_eq = [[0, 1, 1, 0, 0],
          [0, 0, 0, 1, 1]]
rhs_eq = [1, 1]

In [85]:
agg_pass_groups

,total_trips,total_free_blocks,total_15min_blocks_post_free,total_minutes,passes_sold,new_pass_prices,average_charged_blocks_per_ride,percent_total_trips
type_pass,,,,,,,,
single,875156,1103363,310605,10427110.0,0,0,0.354914,0.919076
annualy,24236,28353,5538,801920.0,337,180,0.228503,0.025452
daily,52821,61562,11822,1868049.0,8804,15,0.223812,0.055472


In [86]:
# Revenue for walk ups
walk_up_rev_block = previous_walk_up * agg_pass_groups.loc['single','total_free_blocks']

# Variable income.
trip_rev_day_pass = agg_pass_groups.loc['daily', 'total_15min_blocks_post_free'] * previous_pass_daily
trip_rev_annual_pass = agg_pass_groups.loc['annualy', 'total_15min_blocks_post_free'] * previous_pass_annualy

# Revenue for pass sales is pass is cancelled.
trip_rev_day_no_pass = agg_pass_groups.loc['daily', 'total_free_blocks'] * previous_pass_daily * 0.7
trip_rev_annual_no_pass = agg_pass_groups.loc['annualy', 'total_free_blocks'] * previous_pass_annualy * 0.7

# Revenue for sales of passes.
pass_sales_rev_day = (agg_pass_groups.loc['daily','new_pass_prices'] 
                      * agg_pass_groups.loc['daily','passes_sold'])

pass_sales_rev_annual = (agg_pass_groups.loc['annualy','new_pass_prices'] 
                         * agg_pass_groups.loc['annualy','passes_sold'])

obj_1 = [-walk_up_rev_block, -(trip_rev_day_pass + pass_sales_rev_day), -trip_rev_day_no_pass, -(trip_rev_annual_pass + pass_sales_rev_annual), -trip_rev_annual_no_pass]


In [87]:
obj_1

[-3861770.5, -163979.4, -116352.18000000001, -70628.4, -35724.78]

In [88]:
x0_b = [0,1]
x1_b = [0,1]
x2_b = [0,1]
x3_b = [0,1]
x4_b = [0,1]

In [89]:
from scipy.optimize import linprog

In [90]:
opt_1 = linprog(c = obj_1, A_eq= lhs_eq, b_eq = rhs_eq, method="simplex", bounds = (x0_b, x1_b,x2_b,x3_b,x4_b))

In [91]:
print(opt_1)

     con: array([0., 0.])
     fun: -4096378.3
 message: 'Optimization terminated successfully.'
     nit: 10
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1., 1., 0., 1., 0.])


In [92]:
print("El valor de la función objetivo es ", opt_1.fun * -1)
print("Individual decision_variables:")
print("daily_yes = " , opt_1.x[1])
print("daily_no = ", opt_1.x[2])
print("annualy_yes = ", opt_1.x[3])
print("annualy_no = ", opt_1.x[4])

El valor de la función objetivo es  4096378.3
Individual decision_variables:
daily_yes =  1.0
daily_no =  0.0
annualy_yes =  1.0
annualy_no =  0.0


In [93]:
A = [[1,0, 0, 0, 0],
     [0,1, 1, 0, 0],
     [0,0, 0, 1, 1]]
b = [1, 1, 1]
c_aux_round = [round(num,0) for num in obj_1]
c =[ -x for x in c_aux_round]

In [94]:
import SimplexC

In [95]:
problema = SimplexC.Simplex(c,A,b,problem='Max')
method_result,opt,status = problema.solve()

Solution for x vector, optimization value and status:


In [96]:
print("El valor de la función objetivo es ", (opt) * -1)
print("Individual decision_variables:")
print("daily_yes = " , method_result[1])
print("daily_no = ", method_result[2])
print("annualy_yes = ", method_result[3])
print("annualy_no = ", method_result[4])

El valor de la función objetivo es  4096377.0
Individual decision_variables:
daily_yes =  1.0
daily_no =  0
annualy_yes =  1.0
annualy_no =  0


### Escenario con precios de 2018

In [97]:
agg_pass_groups['new_pass_prices'] = [0, 169, 14.95]
agg_pass_groups['average_charged_blocks_per_ride'] = agg_pass_groups['total_15min_blocks_post_free'] / agg_pass_groups['total_trips']
agg_pass_groups['percent_total_trips'] = agg_pass_groups['total_trips']/sum(agg_pass_groups['total_trips'])

In [98]:
current_tot_rev = (4 * agg_pass_groups['total_15min_blocks_post_free']['daily']
                   + 2.7 * agg_pass_groups['total_15min_blocks_post_free']['single']
                   + 1.8 * agg_pass_groups['total_15min_blocks_post_free']['annualy']
                   + sum(agg_pass_groups['passes_sold'] * agg_pass_groups['new_pass_prices']))
print('Total revenue: $', current_tot_rev)

Total revenue: $ 1084462.7


In [99]:
#Se declaran los precios
previous_walk_up = 3
previous_pass_daily = 4
previous_pass_annualy = 2.5

lhs_eq = [[0, 1, 1, 0, 0],
          [0, 0, 0, 1, 1]]
rhs_eq = [1, 1]

In [100]:
# Revenue for walk ups
walk_up_rev_block = previous_walk_up * agg_pass_groups.loc['single','total_free_blocks']

# Variable income.
trip_rev_day_pass = agg_pass_groups.loc['daily', 'total_15min_blocks_post_free'] * previous_pass_daily
trip_rev_annual_pass = agg_pass_groups.loc['annualy', 'total_15min_blocks_post_free'] * previous_pass_annualy

# Revenue for pass sales is pass is cancelled.
trip_rev_day_no_pass = agg_pass_groups.loc['daily', 'total_free_blocks'] * previous_pass_daily * 0.7
trip_rev_annual_no_pass = agg_pass_groups.loc['annualy', 'total_free_blocks'] * previous_pass_annualy * 0.7

# Revenue for sales of passes.
pass_sales_rev_day = (agg_pass_groups.loc['daily','new_pass_prices'] 
                      * agg_pass_groups.loc['daily','passes_sold'])

pass_sales_rev_annual = (agg_pass_groups.loc['annualy','new_pass_prices'] 
                         * agg_pass_groups.loc['annualy','passes_sold'])

obj_1 = [-walk_up_rev_block, -(trip_rev_day_pass + pass_sales_rev_day), -trip_rev_day_no_pass, -(trip_rev_annual_pass + pass_sales_rev_annual), -trip_rev_annual_no_pass]


In [101]:
x0_b = [0,1]
x1_b = [0,1]
x2_b = [0,1]
x3_b = [0,1]
x4_b = [0,1]

In [102]:
from scipy.optimize import linprog

In [103]:
opt_1 = linprog(c = obj_1, A_eq= lhs_eq, b_eq = rhs_eq, method="simplex", bounds = (x0_b, x1_b,x2_b,x3_b,x4_b))

In [104]:
print(opt_1)

     con: array([0., 0.])
     fun: -3559794.8
 message: 'Optimization terminated successfully.'
     nit: 10
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1., 1., 0., 1., 0.])


In [105]:
print("El valor de la función objetivo es ", opt_1.fun * -1)
print("Individual decision_variables:")
print("daily_yes = " , opt_1.x[1])
print("daily_no = ", opt_1.x[2])
print("annualy_yes = ", opt_1.x[3])
print("annualy_no = ", opt_1.x[4])

El valor de la función objetivo es  3559794.8
Individual decision_variables:
daily_yes =  1.0
daily_no =  0.0
annualy_yes =  1.0
annualy_no =  0.0


In [106]:
A = [[1,0, 0, 0, 0],
     [0,1, 1, 0, 0],
     [0,0, 0, 1, 1]]
b = [1, 1, 1]
c_aux_round = [round(num,0) for num in obj_1]
c =[ -x for x in c_aux_round]

In [107]:
import SimplexC

In [108]:
problema = SimplexC.Simplex(c,A,b,problem='Max')
method_result,opt,status = problema.solve()

Solution for x vector, optimization value and status:


In [109]:
print("El valor de la función objetivo es ", (opt) * -1)
print("Individual decision_variables:")
print("daily_yes = " , method_result[1])
print("daily_no = ", method_result[2])
print("annualy_yes = ", method_result[3])
print("annualy_no = ", method_result[4])

El valor de la función objetivo es  3559795.0
Individual decision_variables:
daily_yes =  1.0
daily_no =  0
annualy_yes =  1.0
annualy_no =  0


### Escenario con propuesta de precios 2020

In [110]:
agg_pass_groups['new_pass_prices'] = [0, 180, 5]
agg_pass_groups['average_charged_blocks_per_ride'] = agg_pass_groups['total_15min_blocks_post_free'] / agg_pass_groups['total_trips']
agg_pass_groups['percent_total_trips'] = agg_pass_groups['total_trips']/sum(agg_pass_groups['total_trips'])

In [111]:
current_tot_rev = (4 * agg_pass_groups['total_15min_blocks_post_free']['daily']
                   + 2.7 * agg_pass_groups['total_15min_blocks_post_free']['single']
                   + 1.8 * agg_pass_groups['total_15min_blocks_post_free']['annualy']
                   + sum(agg_pass_groups['passes_sold'] * agg_pass_groups['new_pass_prices']))
print('Total revenue: $', current_tot_rev)

Total revenue: $ 1000569.9


In [112]:
#Se declaran los precios
previous_walk_up = 3.5
previous_pass_daily = 2.7
previous_pass_annualy = 1.8

lhs_eq = [[0, 1, 1, 0, 0],
          [0, 0, 0, 1, 1]]
rhs_eq = [1, 1]

In [113]:
# Revenue for walk ups
walk_up_rev_block = previous_walk_up * agg_pass_groups.loc['single','total_free_blocks']

# Variable income.
trip_rev_day_pass = agg_pass_groups.loc['daily', 'total_15min_blocks_post_free'] * previous_pass_daily
trip_rev_annual_pass = agg_pass_groups.loc['annualy', 'total_15min_blocks_post_free'] * previous_pass_annualy

# Revenue for pass sales is pass is cancelled.
trip_rev_day_no_pass = agg_pass_groups.loc['daily', 'total_free_blocks'] * previous_pass_daily * 0.7
trip_rev_annual_no_pass = agg_pass_groups.loc['annualy', 'total_free_blocks'] * previous_pass_annualy * 0.7

# Revenue for sales of passes.
pass_sales_rev_day = (agg_pass_groups.loc['daily','new_pass_prices'] 
                      * agg_pass_groups.loc['daily','passes_sold'])

pass_sales_rev_annual = (agg_pass_groups.loc['annualy','new_pass_prices'] 
                         * agg_pass_groups.loc['annualy','passes_sold'])

obj_1 = [-walk_up_rev_block, -(trip_rev_day_pass + pass_sales_rev_day), -trip_rev_day_no_pass, -(trip_rev_annual_pass + pass_sales_rev_annual), -trip_rev_annual_no_pass]


In [114]:
x0_b = [0,1]
x1_b = [0,1]
x2_b = [0,1]
x3_b = [0,1]
x4_b = [0,1]

In [115]:
from scipy.optimize import linprog

In [116]:
opt_1 = linprog(c = obj_1, A_eq= lhs_eq, b_eq = rhs_eq, method="simplex", bounds = (x0_b, x1_b,x2_b,x3_b,x4_b))

In [117]:
print(opt_1)

     con: array([0., 0.])
     fun: -4048751.08
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1., 0., 1., 1., 0.])


In [118]:
print("El valor de la función objetivo es ", opt_1.fun * -1)
print("Individual decision_variables:")
print("daily_yes = " , opt_1.x[1])
print("daily_no = ", opt_1.x[2])
print("annualy_yes = ", opt_1.x[3])
print("annualy_no = ", opt_1.x[4])

El valor de la función objetivo es  4048751.08
Individual decision_variables:
daily_yes =  0.0
daily_no =  1.0
annualy_yes =  1.0
annualy_no =  0.0


In [119]:
A = [[1,0, 0, 0, 0],
     [0,1, 1, 0, 0],
     [0,0, 0, 1, 1]]
b = [1, 1, 1]
c_aux_round = [round(num,0) for num in obj_1]
c =[ -x for x in c_aux_round]

In [120]:
import SimplexC

In [121]:
problema = SimplexC.Simplex(c,A,b,problem='Max')
method_result,opt,status = problema.solve()

Solution for x vector, optimization value and status:


In [122]:
print("El valor de la función objetivo es ", (opt)*-1)
print("Individual decision_variables:")
print("daily_yes = " , method_result[1])
print("daily_no = ", method_result[2])
print("annualy_yes = ", method_result[3])
print("annualy_no = ", method_result[4])

El valor de la función objetivo es  4048750.0
Individual decision_variables:
daily_yes =  0
daily_no =  1.0
annualy_yes =  1.0
annualy_no =  0


In [ ]:
end = time.time()
print(end - start)